In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_hub as hub

In [2]:
# import the training dataset
train = pd.read_json('data/squad-v1.1/train-v1.1.json')

# import the validation dataset
valid = pd.read_json('data/squad-v1.1/dev-v1.1.json')

train.head(5)

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1
3,"{'title': 'Genocide', 'paragraphs': [{'context...",1.1
4,"{'title': 'Antibiotics', 'paragraphs': [{'cont...",1.1


In [3]:
train.iloc[2,0]['paragraphs'][0]

{'context': 'Montana i/mɒnˈtænə/ is a state in the Western region of the United States. The state\'s name is derived from the Spanish word montaña (mountain). Montana has several nicknames, although none official, including "Big Sky Country" and "The Treasure State", and slogans that include "Land of the Shining Mountains" and more recently "The Last Best Place". Montana is ranked 4th in size, but 44th in population and 48th in population density of the 50 United States. The western third of Montana contains numerous mountain ranges. Smaller island ranges are found throughout the state. In total, 77 named ranges are part of the Rocky Mountains.',
 'qas': [{'answers': [{'answer_start': 112,
     'text': 'Spanish word montaña (mountain)'}],
   'question': "Where does the state's name come from?",
   'id': '5733bd9bd058e614000b6199'},
  {'answers': [{'answer_start': 370, 'text': '4th'}],
   'question': 'What is the states rank in size?',
   'id': '5733bd9bd058e614000b619a'},
  {'answers':

In [4]:
text = train.iloc[2,0]['paragraphs'][0]['context']

nlp = spacy.load('en_core_web_md')

# get rid of problem chars
text = text.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ')
# a quick way of removing excess whitespace
text = ' '.join(text.split())
doc = nlp(text)

sentences = []
for sentence in doc.sents:
    if len(sentence) > 1:
        # tokenize into sentences
        sentences.append(sentence.string.strip())

print(len(sentences))
print(sentences)


7
['montana i/mɒnˈtænə/ is a state in the western region of the united states.', "the state's name is derived from the spanish word montaña (mountain).", 'montana has several nicknames, although none official, including "big sky country" and "the treasure state", and slogans that include "land of the shining mountains" and more recently "the last best place".', 'montana is ranked 4th in size, but 44th in population and 48th in population density of the 50 united states.', 'the western third of montana contains numerous mountain ranges.', 'smaller island ranges are found throughout the state.', 'in total, 77 named ranges are part of the rocky mountains.']


In [5]:
# loading the model
url = "https://tfhub.dev/google/elmo/3"
embed = hub.Module(url, trainable=True)

In [6]:
# This tells the model to run through th 'sentences' list and return the default output (1024 dimension sentence vectors).

embeddings = embed(sentences, signature='default', as_dict=True)['default']

# Start a session and run ELMo to return the embeddings in variable x

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    context = sess.run(embeddings)

# Number of sentences in the context
print(len(context))
# Embeddings for the context
print(context)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
7
[[ 0.40833336  0.532851    0.26350272 ... -0.23664832  0.9220655
  -0.00500718]
 [ 0.20504235  0.18236613  0.18386517 ... -0.4490636   0.74794257
   0.05367463]
 [ 0.21802989  0.12451639  0.10218335 ... -0.40154085  0.6026789
   0.04383117]
 ...
 [ 0.28803018  0.57629514  0.10492004 ... -0.21173494  1.0415319
   0.39167637]
 [ 0.4019009   0.1932592   0.18722023 ... -0.02434905  0.13864133
  -0.08316532]
 [ 0.20220126  0.1497484   0.06251618 ... -0.2187055   0.3307815
   0.09774988]]


In [7]:
question = "Where does the state's name come from?"
q_list = []
q_list.append(question)

embeddings = embed(q_list, signature='default', as_dict=True)['default']

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    ques = sess.run(embeddings)

print(len(ques))
print(ques)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
1
[[ 0.28262904  0.12829931 -0.0350394  ... -0.13445887  0.46168175
   0.09416606]]


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

score = []
index = 0

for i in context:
    statement = []
    statement.append(i)
    value = cosine_similarity(ques, statement)[0][0]
    answers = []
    answers.append(value)
    answers.append(index)
    index += 1
    score.append(answers)

score.sort()
e = score[-1][1]
answer = sentences[e]
print(answer)

the state's name is derived from the spanish word montaña (mountain).


In [9]:
import nltk
from nltk import Tree
import spacy

def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

en_nlp = spacy.load('en_core_web_md')
[to_nltk_tree(sent.root).pretty_print() for sent in en_nlp(question).sents]

come                
  ________|_______________    
 |    |        name       |  
 |    |         |         |   
 |    |       state      from
 |    |    _____|____     |   
does  ?  the         's Where



[None]

In [10]:
[to_nltk_tree(sent.root).pretty_print() for sent in en_nlp(answer).sents]

derived                             
  ___________________|___________                      
 |   |        |                 from                  
 |   |        |                  |                     
 |   |        |                 word                  
 |   |        |             _____|___________          
 |   |       name          |     |        montaña     
 |   |        |            |     |           |         
 |   |      state          |     |        mountain    
 |   |    ____|______      |     |      _____|______   
 is  .  the          's   the spanish  (            ) 



[None]